In [4]:
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
import torch

In [5]:
?FashionMNIST

In [30]:
datadir="data"
train_data = FashionMNIST(root=datadir, train=True, download=True, transform=ToTensor())
test_data = FashionMNIST(root=datadir, train=False, download=True, transform=ToTensor())

train_loader = torch.utils.data.DataLoader(train_data,batch_size=32)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1)

In [8]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}


In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

def log_layer_weights(model):
  print(f"conv1: {model.conv1.weight.shape}")
  print(f"conv2: {model.conv2.weight.shape}")
  print(f"fc1: {model.fc1.weight.shape}")
  print(f"fc1: {model.fc2.weight.shape}")


def train( model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    log_layer_weights(model)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [33]:
LR=0.001
GAMMA=0.7
EPOCHS=2
device="cuda"

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=LR)


In [34]:


scheduler = StepLR(optimizer, step_size=1, gamma=GAMMA)
for epoch in range(1, EPOCHS + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()


torch.save(model.state_dict(), "mnist_cnn.pt")

conv1: torch.Size([32, 1, 3, 3])
conv2: torch.Size([64, 32, 3, 3])
fc1: torch.Size([128, 9216])
fc1: torch.Size([10, 128])
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298222
Train Epoch: 1 [320/60000 (1%)]	Loss: 2.282948
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.288045
Train Epoch: 1 [960/60000 (2%)]	Loss: 2.324362
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.292611
Train Epoch: 1 [1600/60000 (3%)]	Loss: 2.286153
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.314161
Train Epoch: 1 [2240/60000 (4%)]	Loss: 2.290479
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.291435
Train Epoch: 1 [2880/60000 (5%)]	Loss: 2.329471
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.278800
Train Epoch: 1 [3520/60000 (6%)]	Loss: 2.276107
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.270518
Train Epoch: 1 [4160/60000 (7%)]	Loss: 2.274452
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.284772
Train Epoch: 1 [4800/60000 (8%)]	Loss: 2.287857
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.291194
Train Epoch: 1 [5440/60000 (9%)]	Loss: 2.302172
Train Epoch: 1 [576